![figure](../lab9/lab9_figures/politecnico_h-01.png)
# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork9 -  Define APIs to Custom Overlays**

________________


## Introduction ##
In this tutorial you will learn how to make custom overlays and use them in PYNQ. This notebook can be uploaded to the PYNQ board and you can run it from there.


### Objectives ###
After completing this lab, you will be able to:

* Create Overlays in Vivado with custom hardware;
* Create APIs in Python for your GPIO or AXI custom hardware;
* Transfer data between PS and PL using GPIO and AXI;


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`

This tutorial was inspired in [Cathal McCabe](https://www.youtube.com/watch?v=UBsCNPWudww) and [Yun Qu](https://www.youtube.com/watch?v=RPTuhVeoGTI) videos, and makes use of HDL code taken from [myhdl](https://www.myhdl.org/docs/examples/jc2.html) and [allaboutfpga](https://allaboutfpga.com/vhdl-code-for-clock-divider/).

________________


## Step 1 - Create a Vivado Overlay and control the IP using GPIO##


### Step 1.1 ###

In this first step you will create a Vivado project with a Zynq processor and a Jhonson counter as the custom RTL module.

* **Open** Vivado and create a new Vivado RTL project in **{labs}** with name **lab9**. Do not specify sources at this time and choose the PYNQ-Z2 board as the default part. Click **Finish**


* **Create Block Design** named **lab9bd** and local to project. Click **OK**.


* Click **Add IP** and add a **ZYNQ7 Processing System** to the design canvas.


* **Run Block Automation** with all the default option. Click **OK**.


* In the *Sources* pan, click on **Add Sources -> Add or create design sources** and add the file **johnson_counter.v** provided in {sources}\lab9. Click **Finish**.


* You should now have this file in your design sources list. **Drag** this file into your Block Design in order to instantiate it, as shown below.


![Figure](../lab9/lab9_figures/fig1.png)


* Select the *jc2_0* output, right click-it and select **Create port**. Change the name to **leds[3:0]**.


* Double-click the *processing_system7_0*. Select the *MIO Configuration* page and then expand **I/O Peripherals -> GPIO** and scheck the box near **EMIO GPIO** with **Width=64**. Click **OK**.


<div class="alert alert-block alert-info">
<b>Note:</b> EMIO is an extended MIO (Multiuse I/O). Both MIO and EMIO are part of the PS, but MIO can be directly connected to the pins of the Zynq chip, and has nothing to do with PL; and EMIO needs to pass the IO of the PL to connect to the pins of the Zynq chip. So, EMIO is the interface between PS and PL.
</div>


![Figure](../lab9/lab9_figures/fig2.png)


Back in the *Block Design* we will need to add a clock divider because the ZYNQ PS output clock (**FCLK_CLK0**) will be too fast to clock the johnson counter (as we want to see the counter's output in the LEDs).We could use a counter IP and a slicer, as done in previous labs, but we will now try a different approach.


* We will use the **clock_divider.vhd** file provided in {sources}/lab9. So add it to the project and drag it to the block design canvas, following the same steps described earlier for the johnson counter. Ignore if you get a warning message when instantiating the module.


* Open the vhdl file and observe the code. Note that this module generates an output clock with period equal to 50000 periods of the input clock. So, if this input clock is 100MHz, the output clock is 2kHz! This division factor is defined as a *Generic* so it can be updated when double-clicking the module in the *Block Design*.


* Connect the clock signals as shown in the figure bellow.


![Figure](../lab9/lab9_figures/fig3.png)


* In the *Block Design* expand the output **GPIO_0** of the *processing_system7_0* and note that there are 3 groups of 64bit signals:
    * a 64 bit input bus (GPIO_I);
    * a 64 bit output bus (GPIO_O);
    * a 64 bit inout bus (GPIO_T);
    
<div class="alert alert-block alert-info">
<b>Note:</b> GPIO_I[0], GPIO_O[0] and GPIO_T[0] actually correspond to only one EMIO. If you only use this EMIO to connect an LED as an output, then you only need to draw GPIO_O[0] on the PL.
</div>


* Add **3 slice IP** blocks to select the specific GPIO wires from the 64 bit bundle to drive the *goleft*, *goright*, and *stop* inputs of our johnson counter module. 


* Give apropriate names to these 3 slice IPs (**left**, **right** and **stop**) and connect their inputs to the **GPIO_O** bus. Their outputs should be connected to the correspondent input of the johnson counter.


* Double-click each slice IP module and configure then according to:
    * slice *left* -> **Din Width=64** and **Din from 0 to 0**
    * slice *right* -> **Din Width=64** and **Din from 1 to 1**
    * slice *stop* -> **Din Width=64** and **Din from 2 to 2**
    

* The processing system AXI port will not be used, so you should remove it to avoid errors during design validation. To do that, **double-click** the ZYNQ processor, select the **PS-PL Configuration** page and remove the check in **AXI Non SecureEnablement -> GP Master AXI Interface -> M AXI GP0 interface**. click **OK**.

![Figure](../lab9/lab9_figures/fig4.png)


* **Validate** the design and **Regenerate** the layout. In the *Sources* pane, select the **lab9bd**, right-click on it and **Create HDL wrapper**. You should have the following final block design and *Sources* pane.


![Figure](../lab9/lab9_figures/fig5.png)


* **Run Synhesis** for this design (it may take a while)) and choose **Open Synthesized Design ** when synthesis completes.


* Now we need to add pin constraints for the LEDs. We could add a **.XDC** file, as done in previous labs, but we will try something different. In the top right corner choose **I/O Planning** view to open the *Package view* and the available pins.


* Choose the pins to be assigned to the 4 LEDs according to the information provided by **TUL** in the Master XDC file (provided in lab1). You can do this by assigning values in the **Package Pin** and **I/O Std** columns, as shown in the figure below.

![Figure](../lab9/lab9_figures/fig6.png)


* **Save** the design and provide a name to the created new XDC file as **pin_constraints**. Click **OK**. Note that this file appears now in the *Constraints* section of the *Sources* pane and has the usual contents.


* **Close** the synthesized design (cross in the blue ribbon) and run **Generate Bitstream**.


### Step 1.2 ##

In this step you will export the overlay and create a Python API to interface with the PL module, through the Processor.


* Connect PYNQ to your computer via USB and to the router via ethernet cable. Make sure the SD card is inserted and **Turn-On** the board.


* Wait for boot to complete and then connect to the board using any browser of your choice (as explained in lab7).


* Create a folder **lab9** inside the **pynq_labs** folder. Inside create a new folder and name it **bitstream**.


* Using windows explorer go to Vivado lab9 project's folder `{labs}\lab9\lab9.runs\impl_1`, locate the **lab9bd_wrapper.bit** file and upload it to the PYNQ folder **lab9\bitstream**. Rename it simply as **lab9bd.bit**.


* Using windows explorer go to Vivado lab9 project's folder `{labs}\lab9\lab9.srcs\sources_1\bd\lab9bd\hw_handoff`, locate the **lab9bd.hwh** file and upload it to the PYNQ folder **lab9\bitstream**. 


* Go back to Vivado and select **File -> Export -> Export Block Design** and click **OK**. This will create a **lab9bd.tcl** file in your working directory. 


* Make sure you have uploaded all files to ZYNQ folder **lab9\bitstream** (**.bit**, **.tcl** and **.hwh** which should have the same name: **lab9db**).


* In PYNQ, create a new **Python 3** file in **lab9** folder and call it **lab9.ipynb**. Alternatively you can copy this notebook to PYNQ and run the following commands from there.


* Start by importing the overlay, running the following:


In [ ]:
from pynq import Overlay
ol= Overlay("./bitstream/lab9bd.bit")

* Run the next cell to import the GPIO class and map pin numbers.


<div class="alert alert-block alert-info">
<b>Note:</b> The PS GPIO use a Linux kernel module to control the GPIO. This means that the operating system assigns a number to the GPIO at run time. Before using the PS GPIO, the Linux pin number must be mapped to the Python GPIO instance. The **get_gpio_pin()** function which is part of the GPIO class is used to map the PS pin number to the Linux pin number.
</div>

In [ ]:
from pynq import GPIO

left=GPIO(GPIO.get_gpio_pin(0), 'out')
right=GPIO(GPIO.get_gpio_pin(1), 'out')
stop=GPIO(GPIO.get_gpio_pin(2), 'out')

* Now you can control the johnson counter. First you need to set the inputs inactive. If you observe the verilog code of this johnson counter you will see that directions are set with zeros and the stop input is also activated with zeros. So run:

In [ ]:
left.write(1)  #left direction not set
right.write(1) #Right direction not set
stop.write(0)  #The counter is stopped

* To start the counter you must set a direction (set either **left** or **right** input to zero) and write a one to **stop** input:

In [ ]:
left.write(0) #Direction is left
stop.write(1) #The counter is not stopped - it is running

Note that the LEDs have turned on but the frequency is too fast to be observed! Indeed, we have set the clock divider output to oscillate at 2kHz, which is too high for our eyes to see. 

We could go back to Vivado and slow down this clock (divider output)), but it would take long to regenerate the bitstream! Instead we can  reduce the PS output clock frequency!


* Import the class **Clocks** and reduce the PS output clock to 1MHz. Now you can watch the LEDs blink! 

In [ ]:
from pynq import Clocks
Clocks.fclk0_mhz=1
print(f'FCLK0:{Clocks.fclk0_mhz:.6f}MHz')

* You can test both directions and then stop the counter!

In [ ]:
right.write(0) #Direction is right
stop.write(1) #The counter is not stopped - it is running

In [ ]:
stop.write(0) #Stop the counter

* **Exit** Vivado.

________________


## Step 2 - Create a Vivado Overlay with an AXI interface IP ##


## Step 2.1 ##

Lets first create a simple adder IP with AXI interface.

* Create a new Vivado RTL Project named **lab9_axi** within {labs}, targeting PYNQ-Z2 board as usual.


![Figure](../lab9/lab9_figures/fig7.png)


* In the *Sources* pane, click **Add sources** and add the verilog file **axi_adder.v** provided in **{sources}\lab9**. This adder has two 4 bit inputs and one 5 bit output, although the **width** parameter is configurable.


* In the *Flow Navigator -> Project Manager* select **Settings**. In the **General** tab make sure the *Target Language* is set to **verilog** and click **OK**.


* Now we have to pack this IP with an AXI interface. To do that, go to **Tools -> Create and Package New IP** and click **Next**.


* Select **Create a new AXI4 peripheral** and click **Next**.


* In the *Peripheral Details* window make the following changes and click **Next**:
    * Name: *axi_adder*
    * Description: *Adder IP with AXI lite interface*
    * IP location: *{labs}/MyIps_Repo/axi_adder_ip 
    
    
* Leave all the defaults in *Add Interfaces* window as we want to create only one, simple 32 bits slave AXI Lite interface. We could configure only 3 registers as we have only 3 signals in the adder (a, b and c), but you can also leave it with 4 registers. Click **Next**.


![Figure](../lab9/lab9_figures/fig8.png)


* Before you finish, check the **Edit IP** box to edit the IP before it is packages to the repository. Click **Finish**.


A new project will open to let you edit your IP. Before we take on any configuration steps we need first to edit the AXI interface verilog file in order to connect the **axi_adder module** to the **AXI_interface module**. Our IP (**axi_adder_v1**) will include both modules as shown in the figure bellow, and we will  have to manually define parameter **WIDTH** in all modules and make the connections.

![Figure](../lab9/lab9_figures/fig15.png)


* In the *Sources* pane, open the **axi_adder_v1_0** hierarchy and double-click to open the file **axi_adder_v1_0_S00_AXI.v**. You have to change a few things here, namelly:
    * Define parameter **WIDTH**
    * Define ports **a**, **b** and **c**
    * Define internal wires (**a_wire**, **b_wire** and **c_wire**)
    * Connect the ports to internal wires
    * Connect the wires to internal registers (**reg0**, **reg1** and **reg2**)


* Between line 6 and 8, define the **WIDTH** parameter, and between lines 17 and 19, define the **user ports**. Complete the code as shown in the figure.

<div class="alert alert-block alert-info">
<b>Note:</b> User ports are the connection between the AXI interface and the adder. For that reason, the adder inputs (**a** and **b**) are the AXI interface outputs and the adder output **c** is the AXI interface input port.
</div>

![Figure](../lab9/lab9_figures/fig9.png)


* Enter the following code lines to define wire buses (called signals in vhdl).


![Figure](../lab9/lab9_figures/fig10.png)


* At the end of the file, assign **c_wire** to the output register and the content of AXI registers to the first bits of the **a_wire** and **b_wire** buses. 


![Figure](../lab9/lab9_figures/fig20.png)

![Figure](../lab9/lab9_figures/fig11.png)


* Now open the top level file **axi_adder_v1_0.v** and add the width parameter at line 7. Also, pass the parameter WIDTH to the AXI interface at line 48.

![Figure](../lab9/lab9_figures/fig12.png)


* In the *Sources* pane, select **Add Design Sources** and add the **axi_adder.v** file. You can locate this file in `{labs}\lab9_axi\lab9_axi.srcs\sources_1\imports\lab9`.


* Now, instantiate the axi_adder in the **axi_adder_v1_0** file and connect it to the AXI slave interface, as shown in the figure.

![Figure](../lab9/lab9_figures/fig13.png)


* Save the both verilog files and note the the hierarchy is now correct. The axi_adder_IP includes the adder and the slave AXI interface, as shown bellow.


![Figure](../lab9/lab9_figures/fig14.png)


* Close the verilog files and **Run Synthesis** to make sure everything was done correctly.



### Step 2.2 ###

Observe now the **Package IP - axi_adder** window, which includes several IP Packaging steps:

1. **Identification**: Information used to identify your IP
2. **Compatibility**: Configure the parts and/or families of Xilinx devices that are compatible with your IP
3. **File Groups**: Individual files for your IP are grouped into specific file groups
4. **Customization Parameters**: Specify the parameters to customize your IP, if any
5. **Ports and Interfaces**: Top-level ports and interfaces for your IP
6. **Addressing and Memory**: Specify the memory-maps or address spaces, if it applies
7. **Customization GUI**: Configure the parameters that appear on each page of the Customization GUI
8. **Review and Package**: Summary of the IP and repackaging


<div class="alert alert-block alert-info">
<b>Note:</b> If for some reasoon you have closed this window you can open it again. Just go to *Sources* window, click *Hierachy tab* and double click **IP-XACT->component.xml**.
</div>

* In **Identification** tab you can add some information according to the figure bollow. Also, make sure the **zynq** family is defined as compatible to this IP in the **Compatibility** tab.

![Figure](../lab9/lab9_figures/fig16.png)


* In the **Customization Parameters** tab, click on the warning message to *Merge changes from Customization Parameters Wizard*. This will add the parameter **WIDTH** to the module's customizable parameters.


Note that the AXI data bus width is 32 and the AXI address bus width is 4. These are default values for the AXI stardard, so they should not be customizable by the user. Also, base and high addresses should not be customizable.


* Right-click **C_S00_AXI_DATA_WIDTH**, choose to **edit** and make it invisible. Repeat the same procedure to **C_S00_AXI_BASEADDR** and **C_S00_AXI_HIGHADDR**.


![Figure](../lab9/lab9_figures/fig17.png)


* Now right-click **C S00 AXI ADDR WIDTH**, choose to **edit**, make it invisible and do not specify range.


* The only parameter you want to be visible to the user is the **WIDTH**. So, edit it and:
    * Make it visible
    * Change the display name to *Number of bits per input*
    * Specify range: Integers, minimum 2 and maximum 16


* See if the module's **Customization GUI** looks correct and all the information in **Review and Package** is ok. 


* In this last tab you can edit the name and location of the IP archive. Click **edit** and simplify the IP name just as **axi_adder_1.0**. click **OK**.


* You can also **Edit packaging settings**, but defaults are usually fine. You can now click the button **Re-Pakage IP**. Once successful, you can close the current Vivado project (*Edit IP project*).




## Step 2.3 ##

Go back to your **lab9_axi** Vivado projet and hook up your IP to the processing system.


* In the *Flow Navigator* choose settings, and then **IP -> Repository**. In that window make sure you have the **MyIps_Repo/axi_adder_1.0** added to the list and click **OK**.


* Create a new *Block Design* with the name **lab9_axi_top** with a ZYNQ7 processing system and a processor system reset. **Run Block Automation** with the default options and then **Run Connection Automation** to make the connections between the blocks.


* **Add IP** and search for the axi_adder in the library. Double-click to add this IP to the design.

![Figure](../lab9/lab9_figures/fig18.png)


* Add an **AXI interconnect** IP to the design, in order to be able to connect the adder with the processor. Customize it to have only one AXI Master and one AXI Slave interface.


* Double click the processor and make sure the master AXI GP0 interface is enabled. In the **PS-PL Configuration** tab, go to **AXI Non Secure Enablement -> GP Master AXI Interface -> M AXI GP0 interface** and make sure the box is checked.


* **Run Connection Automation** and **Regenerate Layout**. You should have the following diagram. Validate the design and **save** it if it passes validation successfuly.


![Figure](../lab9/lab9_figures/fig19.png)


* Create an **HDL Wrapper** for the board design and run **Generate Bitstream**.



## Step 2.4##


* If not aleady, **power-on** the PYNQ board, wait for boot to complete and then connect to the board using any browser of your choice.


* Navigate to folder **pynq_labs/lab9/bitstream** within PYNQ-Z2 Jupyter explorer window.


* Using windows explorer go to Vivado lab9_axi project's folder `{labs}\lab9_axi\lab9_axi.runs\impl_1`, locate the **lab9_axi_top_wrapper.bit** file and upload it to the PYNQ folder **lab9/bitstream**. Rename it simply as **lab9_axi.bit**.


* Using windows explorer go to Vivado lab9_axi project's folder `{labs}\lab9_axi\lab9_axi.srcs\sources_1\bd\lab9_axi_top\hw_handoff`, locate the **lab9_axi_top.hwh** file and upload it to the PYNQ folder **lab9/bitstream**. 


* Go back to Vivado and select **File -> Export -> Export Block Design** and click **OK**. This will create a **lab9_axi_top.tcl** file in your working directory. Upload this file too, with the same name.


* Make sure you have uploaded all files to ZYNQ folder **lab9/bitstream** (**.bit**, **.tcl** and **.hwh** which should have the same name: **lab9_axi**).


* Start by importing the overlay, running the following:

In [ ]:
from pynq import Overlay
ol=Overlay("./bitstream/lab9_axi.bit")

* Use a question mark to find out what is in the overlay.

In [ ]:
ol?


<div class="alert alert-block alert-info">
<b>Loading an Overlay:</b> When and Overlay is loaded using the **pynq.Overlay** function, all of the IP and hierarchies in the overlay will have drivers assigned to them and used to construct an object hierarchy. The IP can then be accessed via attributes on the returned overlay class using the names of the IP and hierarchies in the block diagram.
</div>


* Check what has been exposed in this overlay by runing the IP dictionary. Note that the IP has the name **axi_adder_0**.

In [ ]:
ol.ip_dict


<div class="alert alert-block alert-info">
<b>The Default API:</b> If no driver has been specified for a type of IP then a DefaultIP will be instantiated offering read and write functions to access the IP’s address space and named accessors to any interrupts or GPIO pins connected to the IP.
</div>



* Write some values to inputs **a** and **b**. Note that these inputs are written as 32 bit words because we are using a 32bit AXI interface.

In [ ]:
ol.axi_adder_0.write(0x0 , 1)   # write a=1 in baseaddress
ol.axi_adder_0.write(0x4 , 2)   # write b=2 in baseaddress + 4 bytes

* Read the output **c** from the AXI interface.

In [ ]:
ol.axi_adder_0.read(0x8)   # read c from baseaddress + 8 bytes

* Write sequencial values to inputs **a** and **b** and read the adder output.

In [ ]:
for i in range(10):
    ol.axi_adder_0.write(0x0 , i) 
    ol.axi_adder_0.write(0x4 , i) 
    print(ol.axi_adder_0.read(0x8))

While the **UnknownIP** driver is useful for determining that the IP is working, it is not the most user-friendly API to expose to the eventual end-users of the overlay. Ideally we want to create an IP-specific driver exposing a single **add** function to call the accelerator. 

Custom drivers are created by inheriting from **UnknownIP** and adding a **bindto** class attribute consisting of the IP types the driver should bind to. The constructor of the class should take a single **description** parameter and pass it through to the super class **__init__**. The description is a dictionary containing the address map and any interrupts and GPIO pins connected to the IP.

* Run the following cell:

In [ ]:
from pynq import DefaultIP

class AddDriver(DefaultIP):
    def __init__(self, description):
            super().__init__(description=description)
            
    bindto = ['MEE_EC:user:axi_adder:1.0']
    
    def add(self, a, b):
        self.write(0x00,a)
        self.write(0x04,b)
        return self.read(0x08)

* Now if we reload the overlay and query the help again we can see that our new driver is bound to the IP

In [ ]:
ol = Overlay("./bitstream/lab9_axi.bit")
ol?

* And we can access the same way as before except now our custom driver with an **add** function is created instead of DefaultIP.

In [ ]:
ol.axi_adder_0.add(12,9)

______

## Challenge ##

The Challenge now is to create a SoC design, with a Processor System and the hardware design created in lab4 - Step 3. The idea is to have the PS controlling the filter coeficient set used by the FIR Compiler (replacing the switches) and therefore change the signal sent to the DAC. 

* Open lab4 project and save it as **lab9hlge**.

* Add a ZYNQ PS and use its Fabric clock zero (**FCLK_CLK0**) as input to the clock wizard (inside the **Clock_Synth**). Change it to 125MHz to match the clock frequency you had as input in lab4.

* Use the PS reset (**FCLK_RESET0_N**) as input to the **Clock_Synth**. Note however that this is an active low reset and thus you will have to switch the position of the inverter inside the hierarchical module.

* Replace the input comming from the swithes with the PS GPIO output port.

* Save the Bard Design with a new name and create a new wrapper.

* You should get something like this:

![Figure](../lab9/lab9_figures/fig21.png)

* Make a simple Python program to ask the user to select a filter and check out what happens with ILA or with a scope connected to DA2board!